This notebook shows how to:

*   Fine-tune Phi-3 mini with QLoRA and LoRA
*   Quantize Phi-3 mini with BitsandBytes and GPTQ
*   Run Phi-3 mini with Transformers

Each section of this notebook can be run independently.



# Inference

With Hugging Face's Transformers (16-bit version)

In [2]:
%%bash
pip install -qqq accelerate transformers auto-gptq optimum

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


CalledProcessError: Command 'b'pip install -qqq accelerate transformers auto-gptq optimum\n'' returned non-zero exit status 1.

Using the original model (16-bit version)

It requires 7.4 GB of GPU RAM

In [4]:
import torch
from transformers import AutoTokenizer, AutoModelForCausalLM, set_seed

set_seed(2024)

prompt = "Africa is an emerging economy because"

model_checkpoint = "microsoft/Phi-3-mini-4k-instruct"

tokenizer = AutoTokenizer.from_pretrained(model_checkpoint,trust_remote_code=True)
model = AutoModelForCausalLM.from_pretrained(model_checkpoint,
                                             trust_remote_code=False,
                                             torch_dtype="auto",
                                             device_map="cuda")

inputs = tokenizer(prompt,
                   return_tensors="pt").to("cuda")
outputs = model.generate(**inputs,
                         do_sample=True, max_new_tokens=120)

response = tokenizer.decode(outputs[0], skip_special_tokens=True)

Loading checkpoint shards: 100%|██████████| 2/2 [00:00<00:00,  2.12it/s]


In [5]:
print(response)

Africa is an emerging economy because it is striving towards rapid economic growth and industrialization while attempting to reduce poverty and improve its inhabitants' quality of life. In recent years, the continent has seen significant advancements in technology, infrastructure, and economic development.

Developing African countries like Rwanda, Ethiopia, and Ghana have made considerable progress in urbanization, technology adoption, and economic reforms. With investments in infrastructure such as roads, ports, and airports, these countries are improving their logistics capabilities, facilitating trade, and attracting foreign direct


## Code Generation

In [7]:
import torch
from transformers import AutoTokenizer, AutoModelForCausalLM, set_seed

set_seed(2024)

prompt = "Write a Python code that reads the content of multiple text files and save the result as CSV"

model_checkpoint = "microsoft/Phi-3-mini-4k-instruct"

tokenizer = AutoTokenizer.from_pretrained(model_checkpoint,trust_remote_code=True)
model = AutoModelForCausalLM.from_pretrained(model_checkpoint,
                                             trust_remote_code=False,
                                             torch_dtype="auto",
                                             device_map="cuda")

inputs = tokenizer(prompt,
                   return_tensors="pt").to("cuda")
outputs = model.generate(**inputs,
                         do_sample=True, max_new_tokens=200)

response = tokenizer.decode(outputs[0], skip_special_tokens=True)

Loading checkpoint shards: 100%|██████████| 2/2 [00:00<00:00,  2.16it/s]


In [8]:
print(response)

Write a Python code that reads the content of multiple text files and save the result as CSV.

To accomplish the task of reading content from multiple text files and saving the results into a CSV file, we'll use Python's built-in modules: `os` for interacting with the file system, `csv` for handling CSV files, and `glob` for retrieving files matching a specified pattern. This solution assumes that the files to be processed are in a specific directory or can be identified using a pattern matching. It also assumes that each text file contains lines of content that represent rows of data in our final CSV file.

```python
import os
import csv
import glob

# Define the path to the directory where the text files are located
# and the name of the file where we will store the CSV results.
directory_path = 'path/to/text/files'
output_csv_file = 'data.csv'

# Use glob to retrieve all text files in the directory.
# For this


With Hugging Face's Transformers with the model quantized with GPTQ 4-bit

It requires 2.7 GB of GPU RAM,

# Quantization

Bitsandbytes NF4

In [ ]:
!pip install -qqq --upgrade transformers bitsandbytes accelerate datasets

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 9.0/9.0 MB 10.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 119.8/119.8 MB 11.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 297.6/297.6 kB 37.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 542.0/542.0 kB 35.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 18.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 26.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 19.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 388.9/388.9 kB 43.8 MB/s eta 0:00:00


# Phi-3 Fine-tuning

In [2]:
%%bash
pip -q install huggingface_hub transformers peft bitsandbytes
pip -q install trl xformers
pip -q install datasets
pip install torch>=1.10

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 199.1/199.1 kB 2.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 119.8/119.8 MB 12.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 302.4/302.4 kB 33.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 245.2/245.2 kB 947.5 kB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 222.7/222.7 MB 2.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 542.0/542.0 kB 42.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 102.0/102.0 kB 13.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 779.1/779.1 MB 1.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 176.2/176.2 MB 5.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 168.1/168.1 MB 5.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 9.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 12.3 MB/s eta 0:00:00
     ━━━━

ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
torchaudio 2.2.1+cu121 requires torch==2.2.1, but you have torch 2.3.0 which is incompatible.
torchtext 0.17.1 requires torch==2.2.1, but you have torch 2.3.0 which is incompatible.
torchvision 0.17.1+cu121 requires torch==2.2.1, but you have torch 2.3.0 which is incompatible.


In [15]:
from transformers import AutoModelForCausalLM, AutoTokenizer, TrainingArguments, BitsAndBytesConfig
from huggingface_hub import ModelCard, ModelCardData, HfApi
from datasets import load_dataset
from jinja2 import Template
from trl import SFTTrainer
import yaml
import torch

In [16]:
MODEL_ID = "microsoft/Phi-3-mini-4k-instruct"
NEW_MODEL_NAME = "opus-samantha-phi-3-mini-4k"

In [17]:
DATASET_NAME = "macadeliccc/opus_samantha"
SPLIT = "train"
MAX_SEQ_LENGTH = 2048
num_train_epochs = 1
license = "apache-2.0"
username = "zoumana"
learning_rate = 1.41e-5
per_device_train_batch_size = 4
gradient_accumulation_steps = 1

In [18]:
if torch.cuda.is_bf16_supported():
  compute_dtype = torch.bfloat16
else:
  compute_dtype = torch.float16

In [19]:
model = AutoModelForCausalLM.from_pretrained(MODEL_ID, trust_remote_code=True)
tokenizer = AutoTokenizer.from_pretrained(MODEL_ID, trust_remote_code=True)
dataset = load_dataset(DATASET_NAME, split="train")

EOS_TOKEN=tokenizer.eos_token_id

Loading checkpoint shards: 100%|██████████| 2/2 [00:03<00:00,  1.65s/it]


In [20]:
dataset

Dataset({
    features: ['conversations'],
    num_rows: 3187
})

In [21]:
# Select a subset of the data for faster processing
dataset = dataset.select(range(100))

In [22]:
dataset

Dataset({
    features: ['conversations'],
    num_rows: 100
})

In [44]:
def formatting_prompts_func(examples):
    convos = examples["conversations"]
    texts = []
    mapper = {"system": "system\n", "human": "\nuser\n", "gpt": "\nassistant\n"}
    end_mapper = {"system": "", "human": "", "gpt": ""}
    for convo in convos:
        # print("convo:", convo)
        text = "".join(f"{mapper[(turn := x['from'])]} {x['value']}\n{end_mapper[turn]}" for x in convo)
        texts.append(f"{text}{EOS_TOKEN}")
    return {"text": texts}

dataset = dataset.map(formatting_prompts_func, batched=True)
print(dataset['text'][8])

Map: 100%|██████████| 100/100 [00:00<00:00, 6565.71 examples/s]


user
 What's the difference between permutations and combinations? I always mix them up.

assistant
 No worries, it's a common mix-up! The key difference is that permutations care about the order of arrangement, while combinations don't. Think of permutations as the 'pickier' of the two. 😉

Imagine you have a set of letters: A, B, and C.

With permutations, the order matters. So, ABC and CBA are two different permutations. There are 6 possible permutations: ABC, ACB, BAC, BCA, CAB, and CBA. We calculate this using the formula n! / (n-r)!, where n is the total number of items and r is the number of items being arranged.

With combinations, the order doesn't matter. So, ABC and CBA are considered the same combination. There are only 3 possible combinations: ABC, AB, and AC. We calculate this using the formula n! / (r! * (n-r)!), where n is the total number of items and r is the number of items being chosen.

A helpful trick: If the question mentions words like 'arrange' or 'order,' lean

In [25]:
args = TrainingArguments(
    evaluation_strategy="steps",
    per_device_train_batch_size=7,
    gradient_accumulation_steps=4,
    gradient_checkpointing=True,
    learning_rate=1e-4,
    fp16 = not torch.cuda.is_bf16_supported(),
    bf16 = torch.cuda.is_bf16_supported(),
    max_steps=-1,
    num_train_epochs=3,
    save_strategy="epoch",
    logging_steps=10,
    output_dir=NEW_MODEL_NAME,
    optim="paged_adamw_32bit",
    lr_scheduler_type="linear"
)

/home/sheju347/.conda/envs/LLM/lib/python3.9/site-packages/transformers/training_args.py:1611: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(
huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


In [45]:
from trl import SFTConfig
args = SFTConfig(
    evaluation_strategy="steps",
    per_device_train_batch_size=7,
    gradient_accumulation_steps=4,
    gradient_checkpointing=True,
    learning_rate=1e-4,
    fp16 = not torch.cuda.is_bf16_supported(),
    bf16 = torch.cuda.is_bf16_supported(),
    max_steps=-1,
    num_train_epochs=3,
    save_strategy="epoch",
    logging_steps=10,
    output_dir=NEW_MODEL_NAME,
    optim="paged_adamw_32bit",
    lr_scheduler_type="linear",
    
    dataset_text_field="text",
    max_seq_length=128,
)

/home/sheju347/.conda/envs/LLM/lib/python3.9/site-packages/transformers/training_args.py:1611: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(
huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


In [46]:
trainer = SFTTrainer(
    model=model,
    args=args,
    train_dataset=dataset,
    # dataset_text_field="text",
    # max_seq_length=128,
    # formatting_func=formatting_prompts_func
)

Truncating train dataset: 100%|██████████| 100/100 [00:00<00:00, 11512.06 examples/s]


ValueError: You have set `args.eval_strategy` to steps but you didn't pass an `eval_dataset` to `Trainer`. Either set `args.eval_strategy` to `no` or pass an `eval_dataset`. 

In [28]:
"""
device = 'cuda'
import gc
import os
gc.collect()
torch.cuda.empty_cache()
"""

"\ndevice = 'cuda'\nimport gc\nimport os\ngc.collect()\ntorch.cuda.empty_cache()\n"

In [29]:
trainer.train()

NameError: name 'trainer' is not defined